In [1]:
from sentence_transformers import SentenceTransformer

bge_model = SentenceTransformer("BAAI/bge-m3")

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]
# embeddings = model.encode(sentences)

# similarities = model.similarity(embeddings, embeddings)
# print(similarities.shape)
# # [4, 4]

/home/gleb_galagan/tbank_recsys/base_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import polars as pl
import numpy as np
import typing as t
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from tqdm.notebook import tqdm
from IPython.display import HTML
from dataclasses import dataclass
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize


In [4]:
data_folder = "/home/gleb_galagan/tbank_recsys/sirius_recsys/sirius-2025-recsys/data/"
books = pl.read_parquet(data_folder + "books.pq")
print("Books:")
print(books.shape)
display(books.head(5))

Books:
(34322, 8)


item_id,series,tags,title,description,url,image_url,authors
i64,list[str],list[str],str,str,str,str,list[struct[2]]
0,[null],"[""e-book"", ""young-adult"", … ""y-a""]","""Hallie Hath No Fury . . .""","""There are two sides to every s…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""1879494"",""""}]"
1,"[""149079""]","[""primary"", ""melissa-j--morgan"", … ""fiction""]","""Hide and Shriek: Super Special…","""The girls go on an overnight a…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""21740"",""""}]"
2,[null],"[""friendship"", ""middle-reader"", … ""my-library""]","""Dear Mom, You're Ruining My Li…","""Samantha Slayton worries about…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""18946"",""""}]"
3,"[""151088""]","[""summer-2017"", ""bullying"", … ""re-read""]","""Bratfest at Tiffany's (Clique …","""Massie Block: The Briarwood bo…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""4605"",""""}]"
4,"[""812067""]","[""rosemary-vernon"", ""young-adult"", … ""to-read""]","""Questions of Love (Sweet Dream…","""When Sammi Edwards is chosen t…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""792676"",""""}]"


In [6]:
# books_emb = books.with_columns(bge_model.encode(pl.col('title')).alias('title_embeddings'))
# books_emb
# Get all titles
titles = books['title'].to_list()

# Encode all at once (more efficient)
embeddings = bge_model.encode(titles)

# Create new dataframe with embeddings
books_emb = books.with_columns(
    pl.Series("title_embeddings", list(embeddings))
)
books_emb

item_id,series,tags,title,description,url,image_url,authors,title_embeddings
i64,list[str],list[str],str,str,str,str,list[struct[2]],"array[f32, 1024]"
0,[null],"[""e-book"", ""young-adult"", … ""y-a""]","""Hallie Hath No Fury . . .""","""There are two sides to every s…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""1879494"",""""}]","[-0.014391, -0.001098, … 0.043958]"
1,"[""149079""]","[""primary"", ""melissa-j--morgan"", … ""fiction""]","""Hide and Shriek: Super Special…","""The girls go on an overnight a…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""21740"",""""}]","[-0.017193, 0.014635, … 0.068515]"
2,[null],"[""friendship"", ""middle-reader"", … ""my-library""]","""Dear Mom, You're Ruining My Li…","""Samantha Slayton worries about…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""18946"",""""}]","[-0.015779, 0.030699, … 0.000723]"
3,"[""151088""]","[""summer-2017"", ""bullying"", … ""re-read""]","""Bratfest at Tiffany's (Clique …","""Massie Block: The Briarwood bo…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""4605"",""""}]","[-0.034687, 0.014636, … 0.054508]"
4,"[""812067""]","[""rosemary-vernon"", ""young-adult"", … ""to-read""]","""Questions of Love (Sweet Dream…","""When Sammi Edwards is chosen t…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""792676"",""""}]","[-0.027176, 0.023453, … 0.020856]"
…,…,…,…,…,…,…,…,…
34317,"[""849421""]","[""will-never-read"", ""action"", … ""fiction""]","""The Cruelty (The Cruelty, #1)""","""When her diplomat father is ki…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""14713542"",""""}]","[-0.025789, 0.045962, … -0.005911]"
34318,"[""795937""]","[""haven-t-bothered-to-finish"", ""library"", … ""favorites""]","""Does Snogging Count as Exercis…","""As one of the only girls in Ye…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""753459"",""""}]","[-0.032484, -0.004517, … 0.019637]"
34319,"[""921523""]","[""young-adult"", ""200-299-pgs"", … ""i-don-t-know""]","""Discovery (The Gifted, #1)""","""All twenty-three year old Kate…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""14590184"",""""}]","[0.007021, 0.001272, … 0.027752]"


In [9]:
books_emb.write_parquet('books_title_emb.parquet')

# To read it back later:
# books_emb = pl.read_parquet('books_title_emb.parquet')

In [4]:
# data_folder = "/home/gleb_galagan/tbank_recsys/sirius_recsys/sirius-2025-recsys/data/"
books = pl.read_parquet('/home/gleb_galagan/tbank_recsys/baseline_solution/books_title_emb.parquet')
print("Books:")
print(books.shape)
display(books.head(5))

Books:
(34322, 9)


item_id,series,tags,title,description,url,image_url,authors,title_embeddings
i64,list[str],list[str],str,str,str,str,list[struct[2]],"array[f32, 1024]"
0,[null],"[""e-book"", ""young-adult"", … ""y-a""]","""Hallie Hath No Fury . . .""","""There are two sides to every s…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""1879494"",""""}]","[-0.014391, -0.001098, … 0.043958]"
1,"[""149079""]","[""primary"", ""melissa-j--morgan"", … ""fiction""]","""Hide and Shriek: Super Special…","""The girls go on an overnight a…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""21740"",""""}]","[-0.017193, 0.014635, … 0.068515]"
2,[null],"[""friendship"", ""middle-reader"", … ""my-library""]","""Dear Mom, You're Ruining My Li…","""Samantha Slayton worries about…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""18946"",""""}]","[-0.015779, 0.030699, … 0.000723]"
3,"[""151088""]","[""summer-2017"", ""bullying"", … ""re-read""]","""Bratfest at Tiffany's (Clique …","""Massie Block: The Briarwood bo…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""4605"",""""}]","[-0.034687, 0.014636, … 0.054508]"
4,"[""812067""]","[""rosemary-vernon"", ""young-adult"", … ""to-read""]","""Questions of Love (Sweet Dream…","""When Sammi Edwards is chosen t…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""792676"",""""}]","[-0.027176, 0.023453, … 0.020856]"


In [5]:
# books_emb = books_emb.with_columns(bge_model.encode(pl.col('description')).alias('description_embeddings'))
# books_emb
# books_emb = books.with_columns(bge_model.encode(pl.col('title')).alias('title_embeddings'))
# books_emb
# Get all titles
descriptions = books['description'].to_list()

# Encode all at once (more efficient)
embeddings_description = bge_model.encode(descriptions)

# Create new dataframe with embeddings
books_emb = books.with_columns(
    pl.Series("description_embeddings", list(embeddings_description))
)
books_emb

item_id,series,tags,title,description,url,image_url,authors,title_embeddings,description_embeddings
i64,list[str],list[str],str,str,str,str,list[struct[2]],"array[f32, 1024]","array[f32, 1024]"
0,[null],"[""e-book"", ""young-adult"", … ""y-a""]","""Hallie Hath No Fury . . .""","""There are two sides to every s…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""1879494"",""""}]","[-0.014391, -0.001098, … 0.043958]","[0.001658, 0.014927, … -0.007849]"
1,"[""149079""]","[""primary"", ""melissa-j--morgan"", … ""fiction""]","""Hide and Shriek: Super Special…","""The girls go on an overnight a…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""21740"",""""}]","[-0.017193, 0.014635, … 0.068515]","[0.000373, 0.009427, … 0.023978]"
2,[null],"[""friendship"", ""middle-reader"", … ""my-library""]","""Dear Mom, You're Ruining My Li…","""Samantha Slayton worries about…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""18946"",""""}]","[-0.015779, 0.030699, … 0.000723]","[0.015735, 0.015547, … 0.033758]"
3,"[""151088""]","[""summer-2017"", ""bullying"", … ""re-read""]","""Bratfest at Tiffany's (Clique …","""Massie Block: The Briarwood bo…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""4605"",""""}]","[-0.034687, 0.014636, … 0.054508]","[-0.012077, 0.016581, … 0.051061]"
4,"[""812067""]","[""rosemary-vernon"", ""young-adult"", … ""to-read""]","""Questions of Love (Sweet Dream…","""When Sammi Edwards is chosen t…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""792676"",""""}]","[-0.027176, 0.023453, … 0.020856]","[-0.011821, -0.033938, … -0.032632]"
…,…,…,…,…,…,…,…,…,…
34317,"[""849421""]","[""will-never-read"", ""action"", … ""fiction""]","""The Cruelty (The Cruelty, #1)""","""When her diplomat father is ki…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""14713542"",""""}]","[-0.025789, 0.045962, … -0.005911]","[-0.035014, 0.027219, … 0.024303]"
34318,"[""795937""]","[""haven-t-bothered-to-finish"", ""library"", … ""favorites""]","""Does Snogging Count as Exercis…","""As one of the only girls in Ye…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""753459"",""""}]","[-0.032484, -0.004517, … 0.019637]","[-0.041801, 0.033155, … 0.055893]"
34319,"[""921523""]","[""young-adult"", ""200-299-pgs"", … ""i-don-t-know""]","""Discovery (The Gifted, #1)""","""All twenty-three year old Kate…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""14590184"",""""}]","[0.007021, 0.001272, … 0.027752]","[-0.008719, -0.009974, … 0.031754]"


In [7]:
books_emb.write_parquet('books_title_description_emb.parquet')


In [8]:
# data_folder = "/home/gleb_galagan/tbank_recsys/sirius_recsys/sirius-2025-recsys/data/"
books_new = pl.read_parquet('/home/gleb_galagan/tbank_recsys/baseline_solution/books_title_description_emb.parquet')
print("Books:")
print(books_new.shape)
display(books_new.head(5))

Books:
(34322, 10)


item_id,series,tags,title,description,url,image_url,authors,title_embeddings,description_embeddings
i64,list[str],list[str],str,str,str,str,list[struct[2]],"array[f32, 1024]","array[f32, 1024]"
0,[null],"[""e-book"", ""young-adult"", … ""y-a""]","""Hallie Hath No Fury . . .""","""There are two sides to every s…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""1879494"",""""}]","[-0.014391, -0.001098, … 0.043958]","[0.001658, 0.014927, … -0.007849]"
1,"[""149079""]","[""primary"", ""melissa-j--morgan"", … ""fiction""]","""Hide and Shriek: Super Special…","""The girls go on an overnight a…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""21740"",""""}]","[-0.017193, 0.014635, … 0.068515]","[0.000373, 0.009427, … 0.023978]"
2,[null],"[""friendship"", ""middle-reader"", … ""my-library""]","""Dear Mom, You're Ruining My Li…","""Samantha Slayton worries about…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""18946"",""""}]","[-0.015779, 0.030699, … 0.000723]","[0.015735, 0.015547, … 0.033758]"
3,"[""151088""]","[""summer-2017"", ""bullying"", … ""re-read""]","""Bratfest at Tiffany's (Clique …","""Massie Block: The Briarwood bo…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""4605"",""""}]","[-0.034687, 0.014636, … 0.054508]","[-0.012077, 0.016581, … 0.051061]"
4,"[""812067""]","[""rosemary-vernon"", ""young-adult"", … ""to-read""]","""Questions of Love (Sweet Dream…","""When Sammi Edwards is chosen t…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""792676"",""""}]","[-0.027176, 0.023453, … 0.020856]","[-0.011821, -0.033938, … -0.032632]"
